In [ ]:
#!/usr/bin/python3

import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np
import pandas as pd
import os
import time

def telecharger_et_sauver_mensuel_era5(annee, mois, output_dir="./csv_vent_mensuel", pause_sec=60):
    print(f"\n=== Traitement : {annee}-{mois:02d} ===")
    output_csv = os.path.join(output_dir, f"vent_{annee}_{mois:02d}.csv")

    # Sauter si déjà traité
    if os.path.exists(output_csv):
        print(f"⏭️ Déjà existant, on saute : {output_csv}")
        return

    nc_path = f"vent_era5_{annee}_{mois:02d}.nc"
    client = cdsapi.Client()

    try:
        client.retrieve(
            'reanalysis-era5-single-levels',
            {
                "product_type": "reanalysis",
                "variable": [
                    "10m_u_component_of_wind",
                    "10m_v_component_of_wind",
                    "100m_u_component_of_wind",
                    "100m_v_component_of_wind",
                    "10m_u_component_of_neutral_wind",
                    "10m_v_component_of_neutral_wind",
                    "instantaneous_10m_wind_gust"
                ],
                "year": str(annee),
                "month": f"{mois:02d}",
                "day": [f"{d:02d}" for d in range(1, 32)],
                "time": [f"{h:02d}:00" for h in range(24)],
                "format": "netcdf",
                "area": [51.5, -5.5, 41.0, 9.5]  # France métropolitaine
            },
            nc_path
        )
    except Exception as e:
        print(f"❌ Erreur de téléchargement {annee}-{mois:02d} : {e}")
        return

    # Lecture NetCDF directement
    try:
        f = netCDF4.Dataset(nc_path)
    except Exception as e:
        print(f"❌ Erreur d'ouverture du fichier .nc : {e}")
        return

    print("📄 Variables NetCDF :", list(f.variables.keys()))

    try:
        time_var_name = [k for k in f.variables if 'time' in k.lower()][0]
    except IndexError:
        raise KeyError("❌ Aucune variable temporelle détectée dans le NetCDF.")

    times = num2date(f.variables[time_var_name][:], f.variables[time_var_name].units)
    latitudes = f.variables['latitude'][:]
    longitudes = f.variables['longitude'][:]

    # Construction DataFrame de base
    times_grid, lat_grid, lon_grid = np.meshgrid(times, latitudes, longitudes, indexing='ij')
    df = pd.DataFrame({
        'time': [t.isoformat() for t in times_grid.flatten()],
        'latitude': lat_grid.flatten(),
        'longitude': lon_grid.flatten()
    })

    # Lecture des variables météo
    variables = {
        "u10": "u10", "v10": "v10",
        "u100": "u100", "v100": "v100",
        "u10n": "u10n", "v10n": "v10n",
        "gust": "i10fg"
    }

    data_arrays = {}
    for short, var in variables.items():
        if var in f.variables:
            print(f"📦 Lecture : {var}")
            data = f.variables[var][:]
            df[short] = data.flatten()
            data_arrays[short] = data
        else:
            print(f"⚠️ Variable absente : {var}")

    # Calcul des vitesses du vent
    def wind_speed(u, v):
        return np.sqrt(u**2 + v**2)

    if "u10" in data_arrays and "v10" in data_arrays:
        df["wind_speed_10m"] = wind_speed(data_arrays["u10"], data_arrays["v10"]).flatten()
    if "u100" in data_arrays and "v100" in data_arrays:
        df["wind_speed_100m"] = wind_speed(data_arrays["u100"], data_arrays["v100"]).flatten()
    if "u10n" in data_arrays and "v10n" in data_arrays:
        df["wind_speed_10m_neutral"] = wind_speed(data_arrays["u10n"], data_arrays["v10n"]).flatten()

    # Sauvegarde du fichier CSV
    os.makedirs(output_dir, exist_ok=True)
    df.to_csv(output_csv, index=False)
    print(f"✅ Sauvegarde : {output_csv}")

    # Pause pour ne pas surcharger l'API
    time.sleep(pause_sec)


# === Boucle 2022 à 2025 ===
for annee in range(2022, 2026):
    for mois in range(1, 13):
        try:
            telecharger_et_sauver_mensuel_era5(annee, mois)
        except Exception as e:
            print(f"❌ Problème {annee}-{mois:02d} : {e}")



=== Traitement : 2022-01 ===


2025-05-13 07:06:35,439 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 07:06:35,441 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 07:06:36,045 INFO Request ID is a4fc33cf-2644-4ecb-a050-1ed8eef40aa1
2025-05-13 07:06:36,116 INFO status has been updated to accepted
2025-05-13 07:07:52,007 INFO status has been updated to successful


📄 Variables NetCDF : ['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10', 'u100', 'v100', 'u10n', 'v10n', 'i10fg']
📦 Lecture : u10
📦 Lecture : v10
📦 Lecture : u100
📦 Lecture : v100
📦 Lecture : u10n
📦 Lecture : v10n
📦 Lecture : i10fg
✅ Sauvegarde : ./csv_vent_mensuel\vent_2022_01.csv

=== Traitement : 2022-02 ===


2025-05-13 07:09:39,547 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 07:09:39,548 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 07:09:40,188 INFO Request ID is 01f918e0-467e-4e85-bf93-442374930d86
2025-05-13 07:09:40,309 INFO status has been updated to accepted
2025-05-13 07:10:56,165 INFO status has been updated to successful


📄 Variables NetCDF : ['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10', 'u100', 'v100', 'u10n', 'v10n', 'i10fg']
📦 Lecture : u10
📦 Lecture : v10
📦 Lecture : u100
📦 Lecture : v100
📦 Lecture : u10n
📦 Lecture : v10n
📦 Lecture : i10fg
✅ Sauvegarde : ./csv_vent_mensuel\vent_2022_02.csv


...

2025-05-13 08:03:28,815 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 08:03:28,817 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 08:03:29,439 INFO Request ID is 8d15a92e-6654-4141-a649-f5993b4e92ac
2025-05-13 08:03:29,914 INFO status has been updated to accepted
2025-05-13 08:04:07,856 INFO status has been updated to successful


📄 Variables NetCDF : ['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10', 'u100', 'v100', 'u10n', 'v10n', 'i10fg']
📦 Lecture : u10
📦 Lecture : v10
📦 Lecture : u100
📦 Lecture : v100
📦 Lecture : u10n
📦 Lecture : v10n
📦 Lecture : i10fg
✅ Sauvegarde : ./csv_vent_mensuel\vent_2022_07.csv


Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000220E812FA50>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11001] getaddrinfo failed)"))], attempt 1 of 500
Retrying in 120 seconds



=== Traitement : 2022-08 ===


Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000220E812C990>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11001] getaddrinfo failed)"))], attempt 2 of 500
Retrying in 120 seconds
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000220E812DED0>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11001] getaddrinfo failed)"))], attempt 3 of 500
Retrying in 120 seconds
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at